In [2]:
import math
import pylab
%matplotlib inline

In [3]:
def fact(x):
    if x == 0 or x == 1:
        prod = 1
    else:
        prod = x*fact(x-1)
    return prod

objeto dual

In [4]:
class Dual(object):
    def __init__(self,fun,der=0):
        self.fun = fun
        self.der = der
    def __str__(self):
        return "Dual(%s,%s)" % (str(self.fun),str(self.der))    
    def __real__(self, a):
        return Dual(a,0)
    def __add__(self, b):
        if type(self) < type(b):
            b = Dual(b)
        return Dual(self.fun+b.fun,self.der+b.der)
    def __radd__(self, b):
        return self.__add__(b)
    def __sub__(self, b):
        if type(self) < type(b):
            b = Dual(b)
        return Dual(self.fun-b.fun,self.der-b.der)
    def __rsub__(self, b):
        return self.__sub__(b)
    def __mul__(self, b):
        if type(self) < type(b):
            return Dual(b*self.fun,b*self.der)
        return Dual(self.fun*b.fun, (self.fun*b.der) + (b.fun*self.der))
    def __rmul__(self, b):
        return self.__mul__(b)
    def __div__(self, b):
        if type(self) < type(b) and b != 0:
            return Dual(self.fun/float(b), self.der/float(b))
        return Dual(self.fun/b.fun, (self.der - (self.fun/float(b.fun))*b.der)/float(b.fun))
    def __rdiv__(self,b):
        return self.__div__(b)
    def __pow__(self,b):
        return Dual(self.fun**b, b*(self.fun**(b-1))*self.der)

In [5]:
a = Dual(1,2)

Derivada del término lineal

In [6]:
def xdual(x0):
    return Dual(x0,1)

Pruebas

In [7]:
def test1(x):
    return 3*x**3-2

In [8]:
print test1(xdual(1))

Dual(1,9)


In [9]:
def test2(x):
    return (3*x**2-8*x+5)/(7*x**3-1)

In [10]:
print test2(xdual(1))

Dual(0,-0.333333333333)


In [11]:
# Ahora definiremos métodos para funciones
def sqrt(self):
    if type(self) == Dual:
        return Dual(math.sqrt(self.fun), (0.5)*(self.der/math.sqrt(self.fun)))
    elif type(self) == Taylor:
        return self**(0.5)
    else:
        return math.sqrt(self)
def exp(self, nmax=5):
    if type(self) == Dual:
        return Dual(math.exp(self.fun), self.der*math.exp(self.fun))
    elif type(self) == Taylor:
        self = autoPromocion(self,max(gradoMaxTaylor(self),nmax))
        n = gradoMaxTaylor(self)
        exp_t = Taylor([0]*n)
        exp_t.pol[0] = exp(self.pol[0])
        for k in range(1,n):
            suma = 0
            for j in range(k):
                suma += (k-j)*self.pol[k-j]*exp_t.pol[j]
            exp_t.pol[k] = suma*(1./k)
        return exp_t
    else:
        return math.exp(self)
def log(self, nmax=5):
    if type(self) == Dual:
        return Dual(math.log(self.fun), self.der/self.fun)
    elif type(self) == Taylor:
        self = autoPromocion(self,max(gradoMaxTaylor(self),nmax))
        n = gradoMaxTaylor(self) # grado máximo
        L = Taylor([0]*n)
        s = 0 # índice desde donde empezamos
        while self.pol[s] == 0:
            s += 1
        L.pol[s] = log(self.pol[s])
        for k in range(s+1,n):
            suma = 0
            for j in range(s+1,k):
                suma += (j)*L.pol[j]*self.pol[k-j]
            L.pol[k] = (1./self.pol[s])*(self.pol[k]-suma/(k-s))
        return L
    else:
        return math.log(self)
def sin(self,nmax=5):
    if type(self) == Dual:
        return Dual(math.sin(self.fun), self.der*math.cos(self.fun))
    elif type(self) == Taylor:
        self = autoPromocion(self,max(gradoMaxTaylor(self),nmax))
        n = gradoMaxTaylor(self)
        S = Taylor([0]*n)
        for k in range(n):
            S += ((-1.)**k)*(self**(2*k+1))/fact(2*k+1)
        return S
    else:
        return math.sin(self)
def cos(self,nmax=5):
    if type(self) == Dual:
        return Dual(math.cos(self.fun), -self.der*math.sin(self.fun)) 
    elif type(self) == Taylor:
        self = autoPromocion(self,max(gradoMaxTaylor(self),nmax))
        n = gradoMaxTaylor(self)
        C = Taylor([0]*n)
        for k in range(n):
            C += ((-1.)**k)*(self**(2*k))/fact(2*k)
        return C
    else:
        return math.cos(self)
def tan(self):
    if type(self) == Dual:
        return Dual(math.tan(self.fun), self.der/(math.cos(self.fun)**2))
    elif type(self) == Taylor:
        return sin(self)/cos(self)
    else:
        return math.tan(self)
def cot(self):
    if type(self) == Dual:
        return Dual(1./math.tan(self.fun), -1.*self.der/(math.sin(self.fun)**2))
    elif type(self) == Taylor:
        return cos(self)/sin(self)
    else:
        return 1./math.tan(self)
def sec(self):
    if type(self) == Dual:
        return Dual(1./math.cos(self.fun), self.der*math.tan(self.fun)/math.cos(self.fun))
    elif type(self) == Taylor:
        return 1./cos(self)
    else:
        return 1./math.cos(self)
def csc(self):
    if type(self) == Dual:
        return Dual(1./math.sin(self.fun), -1.*self.der/(math.tan(self.fun)*math.sin(self.fun)))
    elif type(self) == Taylor:
        return 1./sin(self)
    else:
        return 1./math.sin(self)

In [12]:
a = Dual(2.0,1.0)
print a, sin(a)

Dual(2.0,1.0) Dual(0.909297426826,-0.416146836547)


In [13]:
print math.cos(2)
sin(a).der == math.cos(2)

-0.416146836547


True

Sí da hasta la 9na cifra decimal.

In [14]:
def test3(x):
    return x**2 + 2

In [15]:
def test4(x):
    return cos(x)+x**6

In [16]:
a = Dual(2,1)
print sin(test4(a))

Dual(0.683100840448,139.55813893)


Comparando con Wolfram sí da.

# Taylor

In [17]:
class Taylor(object):
    def __init__(self, pol):
        self.pol = pol
    def __str__(self):
        return "Taylor(%s)" % (str(self.pol))
    def __add__(self, b):
        if type(b) != (list and Taylor):
            b = Taylor([b])
        nuevoPol = [i+j for i,j in zip(promocionTaylor(self,b).pol,promocionTaylor(b,self).pol)]
        return Taylor(nuevoPol)
    def __radd__(self, b):
        return self.__add__(b)
    def __sub__(self, b):
        if type(b) != (list and Taylor):
            b = Taylor([b])
        nuevoPol = [i-j for i,j in zip(promocionTaylor(self,b).pol,promocionTaylor(b,self).pol)]
        return Taylor(nuevoPol)
    def __rsub__(self, b):
        return self.__sub__(b)
    def __mul__(self, b):
        if type(b) != (list and Taylor):
            b = Taylor([b])
        n = gradoMaxTaylor(self) + gradoMaxTaylor(b) - 1
        r = Taylor([0]*n)
        A = promocionTaylor(self,r)
        B = promocionTaylor(b,r)
        for k in range(n):
            suma = 0
            for j in range(k+1):
                suma += A.pol[j]*B.pol[k-j]
                #print "elemento %d =" % k, suma
            r.pol[k] = suma
        return r
    def __rmul__(self, b):
        return self.__mul__(b)
    def __div__(self, b):
        if type(b) != (list and Taylor):
            b = Taylor([b])
        A = promocionTaylor(self,b)
        B = promocionTaylor(b,self)
        n = gradoMaxTaylor(A)
        r = Taylor([0]*n)
        s = 0 # índice desde donde empezamos
        while B.pol[s] == 0: # checamos si el primer término no es nulo
            s += 1;
        r.pol[0] = float(A.pol[s])/B.pol[s];
        for k in range(s+1,n):
            suma = 0
            for j in range(k):
                suma += r.pol[j]*B.pol[k-j]
            r.pol[k-s] = float(A.pol[k]-suma)/B.pol[s]    
        return r
    def __rdiv__(self,b):
        return self.__div__(b)
    def __pow__(self,n):
        res = Taylor([1.])
        for k in range(n):
            res = res*self
        return res

In [18]:
def gradoMaxTaylor(a,b=Taylor([0])):
    return max(len(a.pol), len(b.pol))

def promocionTaylor(a,b):
    n = gradoMaxTaylor(a,b)-gradoMaxTaylor(a)
    nuevoPol = a.pol[:]
    nuevoPol.extend([0]*n)
    return Taylor(nuevoPol)

def autoPromocion(a,n):
    return promocionTaylor(a,Taylor([0]*n))

In [19]:
a = Taylor([1,3])
b = a*a
c = b*b
print c*a, gradoMaxTaylor(a)+ gradoMaxTaylor(c)
print 

Taylor([1, 15, 90, 270, 405, 243]) 7



In [20]:
a = Taylor([1,3])

print a*a*a*a*a*a*a, a**7

Taylor([1, 21, 189, 945, 2835, 5103, 5103, 2187]) Taylor([1.0, 21.0, 189.0, 945.0, 2835.0, 5103.0, 5103.0, 2187.0])


In [21]:
b = Taylor([9,0,0,-math.pi*math.e])
print log(b)

Taylor([2.1972245773362196, 0.0, 0.0, -0.9488593580748406, 0.0])


In [23]:
print exp(Taylor([1,2]))

Taylor([2.718281828459045, 5.43656365691809, 5.43656365691809, 3.624375771278727, 1.8121878856393634])


In [20]:
texp = Taylor([0,1])
print exp(texp,5)
tlog = Taylor([1,1])
print log(tlog,10)

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664])
Taylor([0.0, 1.0, -0.5, 0.3333333333333333, -0.25, 0.2, -0.16666666666666666, 0.14285714285714285, -0.125, 0.1111111111111111])


In [21]:
a = Taylor([0,0,1,0,0])
b = Taylor([0,0,0,0,1])
print b/a
print a, autoPromocion(a,10)

Taylor([0.0, 0.0, 1.0, 0, 0])
Taylor([0, 0, 1, 0, 0]) Taylor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])


In [22]:
a = Taylor([3,4,0,4])
b = Taylor([1,3])
print a,b
print a+b
print b+a
print b+b+a
print b-a
print b*b
print b/a

Taylor([3, 4, 0, 4]) Taylor([1, 3])
Taylor([4, 7, 0, 4])
Taylor([4, 7, 0, 4])
Taylor([5, 10, 0, 4])
Taylor([-2, -1, 0, -4])
Taylor([1, 6, 9])
Taylor([0.3333333333333333, 0.5555555555555556, -0.7407407407407408, 0.54320987654321])


In [23]:
print b, b**6

Taylor([1, 3]) Taylor([1.0, 18.0, 135.0, 540.0, 1215.0, 1458.0, 729.0])
